[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/repos-especializacion-UdeA/trabajo-final_AA1/blob/main/notebooks_pasos_previos/merge_databases.ipynb)

# Combinación de todas las bases de datos

El siguiente notebook explora de manera sencilla un archivo de matlab donde se guarda la información de un sensor.

In [1]:
try:
    import scipy.io
except ImportError:
    !pip install scipy

## 1. Librerias y configuraciones previas

In [2]:
import sys
import os

# Get the absolute path of the current notebook
notebook_path = "."
print(notebook_path)
try:
    import google.colab
    !git clone https://github.com/repos-especializacion-UdeA/trabajo-final_AA1.git
    %cd /content/trabajo-final_AA1/notebooks_pasos_previos   
    %pwd
    ruta_base = '/content/trabajo-final_AA1/notebooks_pasos_previos/'
    sys.path.append(ruta_base)
except ImportError:
    print("El notebook no se está ejecutando en Google Colab.")
    ruta_base = './'

.
El notebook no se está ejecutando en Google Colab.


In [3]:
# Tratamiento de datos
# ==============================================================================
import pandas as pd
import numpy as np

# Almacenar en caché los resultados de funciones en el disco
# ==============================================================================
import joblib


# Gestion de librerias
# ==============================================================================
from importlib import reload

# Matemáticas y estadísticas
# ==============================================================================
import math

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns


# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

# Formateo y estilo
# ==============================================================================
from IPython.display import Markdown, display

# Biblioteca scipy y componentes
# ==============================================================================
import scipy.io
from scipy import signal

## 2. Funciones

In [4]:
# Funciones externas
# ==============================================================================
from utils.funciones1 import multiple_plot
from utils.funciones2 import test_hola, graficar_medida, graficar_medida2, filter_signal, \
                             segmentar_data_base, aplanar_data_base, aplanar_ventana

In [5]:
test_hola()

Hola amigos


## 3. Carga del dataset

In [6]:
DATA_PATH = "./datasets/"

archivos_mat = ['S1_A1_E1.mat', 
         'S2_A1_E1.mat', 
         'S3_A1_E1.mat', 
         'S4_A1_E1.mat', 
         'S5_A1_E1.mat', 
         'S6_A1_E1.mat', 
         'S7_A1_E1.mat', 
         'S8_A1_E1.mat', 
         'S9_A1_E1.mat', 
         'S10_A1_E1.mat', 
         'S11_A1_E1.mat', 
         'S12_A1_E1.mat', 
         'S13_A1_E1.mat', 
         'S14_A1_E1.mat', 
         'S15_A1_E1.mat', 
         'S16_A1_E1.mat', 
         'S17_A1_E1.mat', 
         'S18_A1_E1.mat', 
         'S19_A1_E1.mat', 
         'S20_A1_E1.mat', 
         'S21_A1_E1.mat', 
         'S22_A1_E1.mat', 
         'S23_A1_E1.mat', 
         'S24_A1_E1.mat', 
         'S25_A1_E1.mat', 
         'S26_A1_E1.mat',
         'S27_A1_E1.mat']

Carga de cada una de las bases de datos asociadas a los sugetos y combinación en una sola.

In [7]:
## Carga de la base de datos
data_base = pd.DataFrame()

for i in range(len(archivos_mat)):
    ruta_archivo_mat = DATA_PATH = "./datasets/" + archivos_mat[i]
    archivo_mat = scipy.io.loadmat(ruta_archivo_mat)
    # Obtencion de las columnas de interes
    df_emg = pd.DataFrame(archivo_mat['emg'])
    df_emg.columns = ['emg_' + str(col + 1) for col in df_emg.columns]
    df_restimulus = pd.DataFrame(archivo_mat['restimulus'])
    df_restimulus.rename(columns={0: 'label'}, inplace= True)
    df_repetition = pd.DataFrame(archivo_mat['rerepetition'])
    df_repetition.rename(columns={0: 'rep'},inplace= True)
    df_subject = pd.DataFrame({'s': [i + 1] * df_repetition.shape[0]}, dtype='int8')
    df_subject =  pd.concat([df_subject, df_emg, df_repetition, df_restimulus], axis=1)
    # print(f"Muestras sujeto {i + 1}: {df_subject.shape[0]}")
    data_base = pd.concat([data_base, df_subject], ignore_index=True)

## Informacion de la base de datos

In [8]:
data_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2731393 entries, 0 to 2731392
Data columns (total 13 columns):
 #   Column  Dtype  
---  ------  -----  
 0   s       int8   
 1   emg_1   float64
 2   emg_2   float64
 3   emg_3   float64
 4   emg_4   float64
 5   emg_5   float64
 6   emg_6   float64
 7   emg_7   float64
 8   emg_8   float64
 9   emg_9   float64
 10  emg_10  float64
 11  rep     uint8  
 12  label   uint8  
dtypes: float64(10), int8(1), uint8(2)
memory usage: 216.2 MB


### Contenido de los primeros registros del dataframe

In [9]:
# Primeros registros
data_base.head(10)

,s,emg_1,emg_2,emg_3,emg_4,emg_5,emg_6,emg_7,emg_8,emg_9,emg_10,rep,label
0,1,0.0684,0.0024,0.0024,0.0024,0.0024,0.0098,0.0024,0.0488,0.0024,0.0342,0,0
1,1,0.0586,0.0024,0.0024,0.0024,0.0024,0.0049,0.0024,0.0415,0.0024,0.0293,0,0
2,1,0.0562,0.0024,0.0024,0.0024,0.0024,0.0049,0.0024,0.0391,0.0024,0.0244,0,0
3,1,0.0562,0.0024,0.0024,0.0024,0.0024,0.0049,0.0024,0.0342,0.0024,0.0171,0,0
4,1,0.0488,0.0024,0.0024,0.0024,0.0024,0.0024,0.0024,0.0366,0.0024,0.0146,0,0
5,1,0.0488,0.0024,0.0024,0.0024,0.0024,0.0024,0.0024,0.0366,0.0024,0.0098,0,0
6,1,0.0439,0.0024,0.0024,0.0024,0.0024,0.0024,0.0024,0.0366,0.0024,0.0098,0,0
7,1,0.0464,0.0024,0.0024,0.0024,0.0024,0.0024,0.0024,0.0366,0.0024,0.0049,0,0
8,1,0.0464,0.0024,0.0049,0.0024,0.0024,0.0024,0.0024,0.0342,0.0024,0.0024,0,0
9,1,0.0415,0.0024,0.0024,0.0024,0.0024,0.0024,0.0024,0.0342,0.0024,0.0024,0,0


## Preprocesamiento de la señal

### Filtrado de ruido

A continuación se realiza un filtrado de ruido de la señal

In [10]:
# perform 2-order 1Hz low-pass filter
emgs_df_filt =  filter_signal(data_base.iloc[:,1:11])  
data_base_filter =  pd.concat([data_base['s'], emgs_df_filt, data_base['rep'] , data_base['label']], axis=1)
data_base_filter.head()

,s,emg_1,emg_2,emg_3,emg_4,emg_5,emg_6,emg_7,emg_8,emg_9,emg_10,rep,label
0,1,0.067523,0.002402,0.002399,0.002403,0.002395,0.009800,0.0024,0.050357,0.0024,0.034192,0,0
1,1,0.066284,0.002402,0.002402,0.002404,0.002395,0.009474,0.0024,0.049707,0.0024,0.032820,0,0
2,1,0.065045,0.002403,0.002405,0.002404,0.002395,0.009149,0.0024,0.049050,0.0024,0.031452,0,0
3,1,0.063810,0.002403,0.002409,0.002405,0.002395,0.008827,0.0024,0.048390,0.0024,0.030091,0,0
4,1,0.062581,0.002404,0.002412,0.002406,0.002395,0.008507,0.0024,0.047726,0.0024,0.028743,0,0


## Segmentacion

Se van a usar ventanas de 30 muestras sobrelapadas en 10 muestras:

In [11]:
numero_de_sujetos = 2
sujetos_muestra = data_base_filter[data_base_filter['s'] <= numero_de_sujetos]
segments = segmentar_data_base(sujetos_muestra, 30, 10)

9369


In [12]:
print(data_base_filter.shape)
print(len(segments))

(2731393, 13)
9369


In [13]:
data_base_to_model = aplanar_data_base(segments)

In [16]:
data_base_to_model.shape

(9369, 302)

In [17]:
data_base_to_model.head(2)

,emg_1_0,emg_2_0,emg_3_0,emg_4_0,emg_5_0,emg_6_0,emg_7_0,emg_8_0,emg_9_0,emg_10_0,...,emg_3_29,emg_4_29,emg_5_29,emg_6_29,emg_7_29,emg_8_29,emg_9_29,emg_10_29,rep,label
0,0.067523,0.066284,0.065045,0.063810,0.062581,0.061362,0.060156,0.058966,0.057793,0.056641,...,0.009423,0.008701,0.008017,0.00737,0.00676,0.006185,0.005646,0.005140,0,0
1,0.045784,0.044986,0.044221,0.043488,0.042789,0.042121,0.041485,0.040880,0.040305,0.039760,...,0.001190,0.001049,0.000926,0.00082,0.00073,0.000654,0.000592,0.000544,0,0


# Modelado

* Puede que sea necesario reducir dimensionalidad usando algunas transformaciones.
* Escalar.
* Normalizar.
* Entrenar y Testear.
* Rezar.

## Referencias

* https://github.com/parasgulati8/NinaPro-Helper-Library
* https://github.com/Lif3line/nina_helper_package_mk2
* https://github.com/cnzero/NinaproCNN/tree/master
* https://github.com/sebastiankmiec/NinaTools
* https://github.com/sun2009ban/divide_NinaPro_database_5
* https://github.com/tsagkas/sEMG-HandGestureRecognition 
* https://repositorio.unbosque.edu.co/items/61d39597-5a61-491c-909a-849e53efe8ad
* https://github.com/parasgulati8/NinaPro-Helper-Library/blob/master/